# TCP协议中的数据传递
## recv()特征
  1. 如果连接断开 ，recv会立即结束阻塞返回空字符串
  2. 当接收缓冲区为空时会阻塞
  3. 如果recv一次接受不完缓冲区内容，下次会继续接受，确保数据不丢失

## send()特征
  1. 如果另一端不存在还试图使用send进行发送则会产生BrokenPipeError异常
  2. 当发送缓冲区满时会阻塞

## 网络收发缓冲区

- **缓冲区的功能：协调读写速度，减少和磁盘交互**  
  recv和send实际上是从缓冲区接受内容，向缓冲区发送内容
  
## tcp粘包
- **产生原因**：tcp传输采用字节流方式，消息之间没有边界，如果发送和接受速度不匹配，会造成多次发的  
  内容被一次接受，形成意义上的误解即粘包
- **产生条件**：当使用send快速的连续发送极有可能产生粘包
- **影响**：如果每次发送的风容代表一个独立的意思，此时产生粘包需要处理。但是如果多次发送的内容本身  
  就是一个连续的整体，此时就不需要处理。
- **如何处理**：
  1. 每次发送后加一个结尾标志，接受端通过标志进行判断
  2. 发送一个数据结构   （如：时间+字节数据+发送的内容，接收端解析（C语言））
  3. 每次发送中间有一个短暂的延迟

# 基于UDP的通信

## 服务端
```python
from socket import *
# 1创建套接字  ---》 数据报套接字
sockfd = socket(AF_INET,SOCK_DGRAM)

# 2.绑定服务端地址
sockfd.bind()

# 3.消息的收发
data,addr = recvfrom(buffersize)
# 功能 ： 接收UDP消息
# 参数 ： 每次最多接收消息的大小
# 返回值 ： data 接收到的内容 
#        addr 接收到的消息的发送者


sendto(data，addr)
# 功能 ： udp发送消息
# 参数 ：  data  要发送的内容  bytes
#       addr  目标地址
# 返回值 ： 发送的字节

# 4.关闭套接字
sockfd.close()

```

### cookie
```python
    # argv.py
    import sys
    print(sys.argv) 
    -----------------------------------------
#     .../day02$ python argv.py
%     ['argv.py']
%     .../day02$ python argv.py 127.0.0.1 8888
#     ['argv.py', '127.0.0.1', '8888']
```
- **功能**：获取来自命令行的参数，形成一个列表 
  - **以空格作为每一项分隔，如果一项中有空格则用引号表示一个整体
  - 命令行内容均作为字符串传入

In [4]:

from socket import *
import sys

# 从命令行传入ip和端口
HOST = sys.argv[1]
PORT = int(sys.argv[2])
ADDR = (HOST,PORT)

# 创建数据报套接字
sockfd = socket(AF_INET,SOCK_DGRAM)

# 绑定地址
sockfd.bind(ADDR)

# 接收消息
data,addr = sockfd.recvfrom(1024)
print('Receive from %s : %s'%(addr,data.decode()))

# 发送消息
sockfd.sendto('收到你的消息'.encode(),addr)

# 关闭套接字
sockfd.close()


In [ ]:
from socket import *
import sys

if len(sys.argv) < 3:
    print('''argv is error !!
            run as 
            python3 udp_cliet.py 127.0.0.1 8888
            ''')
    raise
HOST = sys.argv[1]
PORT = int(sys.argv[2])
SERVER_ADDR = (HOST,PORT)

# 创建套接字
sockfd = socket(AF_INET,SOCK_DGRAM)
while True:
    data = input('消息：')
    if not data:
        
        break
    # 给服务器发送
    sockfd.sendto(data.encode(),SERVER_ADDR)
    data,addr = sockfd.recvfrom(1024)
    print('从服务器收到：',data.decode())
sockfd.close()

## tcp流式套接字和udp数据报套接字区别
1. 流式套接字采用字节流的方式传输数据，而数据报套接字以数据报形式传输
2. tcp会产生粘包现象，udp消息是有边界的不会粘包
3. tcp传输是建立在连接的基础上，保证传输的可靠性，而udp一次接受一个数据报，  
  不保证完整性
4. tcp需要依赖listen accept建立连接，udp不用
5. tcp使用recv  send进行消息收发，udp使用recvfrom和sendto

## 补充： 
- sendall（）  
- 功能参数： 同send 作为tcp消息发送
- 返回值 ： 发送成功返回None  发送失败返回异常

## socket模块和套接字属性
- （s表示一个套接字对象）
- **s.type: 获取套接字类型**
- **s.family: 获取地址族类型**

```python
    from socket import *
    s = socket()
    print(s.type) # SocketKind.SOCK_STREAM
    print(s.family) # AddressFamily.AF_INET
```
- **s.fileno: 获取套接字的文件描述符**
  - 文件描述符：每一个IO操作系统都会为其分配一个不同的  
    正整数，该正整数即为此IO操作的文件描述符。

```python
    from socket import *
    import sys
    s.fileno()   # 1404
    sys.stdin.fileno() # 0
```
- **s.getsockname():获取套接字绑定地址**
- **s.getpeername(): 获取连接端的地址**

```python
    from socket import *
    s = socket(SOL_SOCKET,SO_REUSEADDR,1)
    s.bind(('176.47.9.83',8888))
    print(s.getsockname())  # ('176.47.9.83', 8888)
    s.listen(5)
    c,addr = s.accept()
    print(c.getpeername()) # ('127.0.0.1', 49300)
```

- **s.setsockopt(level,optname,value):设置套接字选项 丰富修改原有套接字功能**
  - 参数：  
       level ; 设置选项的类型  
       optname : 每个选项类型中的子选项
       value: 为选项设置值
- **s.getsockopt(level,optname)：获取套接字选项值**

## udp套接字应用之广播
- 一点发送多点接收
- 目标地址：广播地址，每个网段内最大的地址
      172.60.50.255 ----》 <broadcast>

## tcp应用之HTTP传输
- **http协议 -- 》 超文本传输协议  应用层协议**
- **用途**：网页的获取，基于网站的数据传输基于http协议的数据传输
- -**特点**：
  - 应用层协议，传输层使用tcp传输
  - 简单灵活，和多种语言对接方便
  - 无状态协议，不记录用户的通信内容
  - 成熟稳定 http1.1
- 工作模式：
  - 使用http双方均遵循http协议规定发送
  * 请求方，根据协议组织请求内容发送给对方
  * 服务方，收到内容按照协议解析
  * 服务方，将回复内容按照协议组织发送给请求方
  * 请求方，收到回复根据协议解析

  
